# Compare and Analyze Data on COVID-19 Infections Provided by the [Robert Koch Institute (RKI)](https://www.rki.de/EN/Home/homepage_node.html)

The [Robert Koch Institute (RKI)](https://www.rki.de/EN/Home/homepage_node.html) is the federal government agency responsible for disease control and prevention in Germany. Is publishes data about COVID-19 for all of Germany and uses various channels for that (see [this](https://www.rki.de/EN/Content/infections/epidemiology/outbreaks/COVID-19/COVID19.html) page for an overview).

This notebook has been created for analyzing and comparing data from two different sources that are updated by the RKI daily, but have a different level of detail. The main objective is to understand how the very fine grained numbers provided via github can be aggregated such that they match what is shown in the 
[RKI's COVID-19 dashboard](https://corona.rki.de/).

## Preliminaries

In [1]:
import json

import datetime as dt
import numpy as np
import pandas as pd

from urllib.request import urlopen

In [2]:
# define constants that are used for formatting output 
 
INTEGER_FORMAT_STR = "{:,.0f}"
FOUR_DIGIT_YEAR_FORMAT_STR = "{:.0f}"
LOW_PRECISION_FLOAT_FORMAT_STR = "{:,.1f}"
MEDIUM_PRECISION_FLOAT_FORMAT_STR = "{:,.3f}"
HIGH_PRECISION_FLOAT_FORMAT_STR = "{:,.6f}" 
PERCENT_FORMAT_STR = "{:.2%}"
PER_THOUSAND_FORMATTER = lambda p: f"{p*1000:.2f}\u2030"
DATE_FORMATTER = lambda d: d.strftime("%m-%d")
DUMMY_FORMAT_STR = "{:}"

FORMAT_MAPPER = {"area": INTEGER_FORMAT_STR,
                 "category": DUMMY_FORMAT_STR,
                 "cases": INTEGER_FORMAT_STR,
                 "cases last 7 days": INTEGER_FORMAT_STR,
                 "cases per 100k": LOW_PRECISION_FLOAT_FORMAT_STR,
                 "cases per pop.": PERCENT_FORMAT_STR,
                 "cases last 7 days per 100k": LOW_PRECISION_FLOAT_FORMAT_STR,
                 "deaths": INTEGER_FORMAT_STR,
                 "deaths last 7 days": INTEGER_FORMAT_STR,
                 "deaths per 100k": HIGH_PRECISION_FLOAT_FORMAT_STR,
                 "deaths per pop.": PER_THOUSAND_FORMATTER,
                 "deaths last 7 days per 100k": MEDIUM_PRECISION_FLOAT_FORMAT_STR,
                 "death rate": PERCENT_FORMAT_STR,
                 "death rate last 7 days": PERCENT_FORMAT_STR,
                 "recovered": INTEGER_FORMAT_STR,
                 "recovered last 7 days": INTEGER_FORMAT_STR,
                 "recovered per 100k": LOW_PRECISION_FLOAT_FORMAT_STR,
                 "recovered per pop.": PERCENT_FORMAT_STR,
                 "recovered last 7 days per 100k": LOW_PRECISION_FLOAT_FORMAT_STR,
                 "district name": DUMMY_FORMAT_STR ,
                 "population": INTEGER_FORMAT_STR,
                 "state ID": DUMMY_FORMAT_STR,
                 "state name": DUMMY_FORMAT_STR ,
                 "state pop.": INTEGER_FORMAT_STR,
                 "update time": DATE_FORMATTER,
}

## Load and Analyze Data From the [NPGEO Corona Hub 2020](https://npgeo-corona-npgeo-de.hub.arcgis.com/)

For all German districts, up-to-date COVID-19 data is available via [this](https://opendata.arcgis.com/datasets/917fc37a709542548cc3be077a786c17_0) page. This data appears to be the basis for the [COVID-19 dashboard](https://corona.rki.de/).

In [3]:
# URL for loading data as comma-seperated-data-file

RKI_ARCGIS_URL = "https://opendata.arcgis.com/datasets/917fc37a709542548cc3be077a786c17_0.csv"

In [4]:
# dictionary for renaming and harmonizing column names 

RKI_ARCGIS_COLUMN_NAME_MAPPER = {"cases7_per_100k": "cases last 7 days per 100k",
                                 "cases7_lk" : "cases last 7 days",
                                 "county": "district name",
                                 "cases_per_100k": "cases per 100k",
                                 "cases_per_population": "cases per pop.",
                                 "cases": "cases",
                                 "deaths": "deaths",
                                 "death_rate": "death rate",
                                 "death7_lk": "deaths last 7 days",
                                 "EWZ": "population",
                                 "BL_ID": "state ID",
                                 "EWZ_BL": "state pop.",
                                 "last_update": "update time",
                                 "RS": "district ID",
                                 "BEZ": "category",
                                 "GEN": "municipality name",
                                 "BL": "state name",
                                 "KFL": "area",
}

RKI_ARCGIS_VALUE_CONVERTERS = {"last_update": lambda s: dt.datetime.strptime(s, "%d.%m.%Y, %H:%M Uhr")}

### Read Data

In [5]:
# load main data, but restrict the created dataframe to the most relevant columns 

RKI_ARCGIS_COVID_BY_DISTRICT = \
    pd.read_csv(RKI_ARCGIS_URL, usecols=list(RKI_ARCGIS_COLUMN_NAME_MAPPER.keys()),
                converters=RKI_ARCGIS_VALUE_CONVERTERS)\
        .rename(columns=RKI_ARCGIS_COLUMN_NAME_MAPPER)\
        .sort_values(by="district ID")\
        .set_index("district ID")

# add column for the number of deaths within the last seven days adjusted to a population size of 100.000 people

RKI_ARCGIS_COVID_BY_DISTRICT["deaths last 7 days per 100k"] = \
    10**5 * RKI_ARCGIS_COVID_BY_DISTRICT["deaths last 7 days"] \
    / RKI_ARCGIS_COVID_BY_DISTRICT["population"]

In [6]:
# print date of most recent entry in the loaded data

print(RKI_ARCGIS_COVID_BY_DISTRICT["update time"].max().strftime("last update is from %Y-%m-%d"))

last update is from 2021-09-20


## Load and Analyze RKI's Data on COVID-19 From [GitHub](https://github.com/robert-koch-institut/SARS-CoV-2_Infektionen_in_Deutschland)

Repository ["SARS-CoV-2 Infektionen in Deutschland"](https://github.com/robert-koch-institut/SARS-CoV-2_Infektionen_in_Deutschland) (SARS-CoV-2 Infections in Germany) contains
up-to-date numbers of COVID-19 cases in Germany. The data appears to be what is reported by the districts to the RKI as it lists new cases based on the reporting date, beginning of the disease (reference date), age group, sex and district.

In [Readme.md](https://github.com/robert-koch-institut/SARS-CoV-2_Infektionen_in_Deutschland/blob/master/Readme.md), an explanation for the data is provided (in German). 

In [7]:
# define constants for the URLs that give access to data

RKI_GITHUB_REPOSITORY_URL = "https://github.com/robert-koch-institut/SARS-CoV-2_Infektionen_in_Deutschland"
RKI_GITHUB_RAW_DATA_BASE_URL = RKI_GITHUB_REPOSITORY_URL + "/raw/master"
RKI_GITHUB_ZENODO_REL_URL = "/.zenodo.json"
RKI_GITHUB_COVID_INFECTIONS_REL_URL = "/Aktuell_Deutschland_SarsCov2_Infektionen.csv"

In [8]:
# dictionary for renaming and harmonizing column names 

RKI_GITHUB_COLUMN_NAME_MAPPER = {
    "IdLandkreis": "district ID", 
    "Gemeindename": "municipality name", 
    "Flaeche": "area", 
    "EW_insgesamt": "population",
    "EW_maennlich": "population male", 
    "EW_weiblich" : "population female",
    "Altersgruppe": "age group", 
    "Geschlecht": "sex", 
    "Meldedatum": "reporting date", 
    "Refdatum": "reference date",
    "IstErkrankungsbeginn": "is start of desease", 
    "NeuerFall": "is new case", 
    "NeuerTodesfall": "is new death", 
    "NeuGenesen": "is new recovered",
    "AnzahlFall": "cases", 
    "AnzahlTodesfall": "deaths", 
    "AnzahlGenesen": "recovered",
}

unknown_converter = lambda s : "unknown" if s == "unbekannt" else s

RKI_GITHUB_VALUE_CONVERTERS = {
    "Altersgruppe": unknown_converter,
    "Geschlecht": lambda s: "F" if s == "W" else unknown_converter(s), 
}

# dictionary for setting the date type for some of the columns

RKI_GITHUB_COLUMN_TYPES_MAPPER = {
    "district ID": "int16",
    "reporting date": "datetime64", 
    "reference date": "datetime64",
    "is start of desease": "int32", 
    "is new case": "int8", 
    "is new death": "int8", 
    "is new recovered": "int8",
}

### Read Metadata

The RKI publishes metadata based on [zenodo's](https://about.zenodo.org/) JSON format. Here, it is used to detect the publication date of the data. Typically, this is shortly after 3 AM of the current day (local time in Germany). 

In [9]:
RKI_GITHUB_METADATA = json.loads(urlopen(RKI_GITHUB_RAW_DATA_BASE_URL + RKI_GITHUB_ZENODO_REL_URL).read())
RKI_GITHUB_PUBLICATION_DATE = pd.to_datetime(RKI_GITHUB_METADATA["publication_date"])
print(f"publication date is {RKI_GITHUB_PUBLICATION_DATE:%Y-%m-%d %H:%M:%S%z}")

publication date is 2021-09-20 03:17:25+0200


### Read Data

Load data describing COVID-19 infections and deaths etc.

In [10]:
RKI_GITHUB_COVID_INFECTIONS = pd.read_csv(RKI_GITHUB_RAW_DATA_BASE_URL + RKI_GITHUB_COVID_INFECTIONS_REL_URL, 
                                    converters=RKI_GITHUB_VALUE_CONVERTERS)\
                            .rename(columns=RKI_GITHUB_COLUMN_NAME_MAPPER)\
                            .astype(RKI_GITHUB_COLUMN_TYPES_MAPPER)

### Trying to Understand [Readme.md](https://github.com/robert-koch-institut/SARS-CoV-2_Infektionen_in_Deutschland/blob/master/Readme.md)

According to the text in file [Readme.md](https://github.com/robert-koch-institut/SARS-CoV-2_Infektionen_in_Deutschland/blob/master/Readme.md), are the values for the number of 
infected (column `cases` - `AnzahlFall` in the original data), deceased (column `deaths` - column `AnzahlTodesfall` before renaming) and recovered 
(column `recovered` - `AnzahlGenesen` before renaming) people, **[natural numbers](https://en.wikipedia.org/wiki/Natural_number)** (i.e. elements of {1,2,3 ...}). 

However, negative values seem to occur in the data. 

In [11]:
# show that there are rows, for which the value in one of the columns "cases", "deaths" or "recovered" is negative

((RKI_GITHUB_COVID_INFECTIONS["cases"] < 0) |  \
    (RKI_GITHUB_COVID_INFECTIONS["deaths"] < 0) |  \
        (RKI_GITHUB_COVID_INFECTIONS["recovered"] < 0))\
            .value_counts()[True] > 0

True

Hence, despite what's stated in `Readme.md`, values of the respective columns are **integers** (i.e. elements of {... -3, -2, -1, 0,1,2,3 ...}) and **not** natural numbers. 

If one wants to determine to total number of COVID-19 cases reported in Germany, `Readme.md` seems to imply that only such rows should be considered for which the value 
column `is new case` (`NeuerFall` in the original data) is not 0, because 0 indicates that the respective case was already reported previously. 

Additionally, one would think that rows for which the values in column `is new case` is -1 should be subtracted, because this value indicates that cases have been falsely reported in the past.

However, it appears that 
$$

 n = -1  \Leftrightarrow c < 0
$$ 
if $n$ stands for the value in column `is new case` and $c$ for the value in column `cases` in a row. This means that values for such rows should **not be subtracted but added** (since thy are negative).

In [12]:
# verify that in all rows of dataframe RKI_GITHUB_COVID_INFECTIONS, the value in column "is new case" == -1 iff the value in column "cases" is < 0

equiv = lambda a,b: ((~a) | b) & ((~b) | a) 
equiv(RKI_GITHUB_COVID_INFECTIONS["is new case"] == -1, RKI_GITHUB_COVID_INFECTIONS["cases"] < 0).all()

True

Hence, it should be correct to simply add all values of column `cases`, where `is new case` is not 0 to get the total number of cases in Germany ...

In [13]:
# print the sum for column "cases", filtered by "is new case" != 0 (i.e. is new case) 

print(f'{RKI_GITHUB_COVID_INFECTIONS.loc[RKI_GITHUB_COVID_INFECTIONS["is new case"] != 0, "cases"].sum():,d}')

3,736


... but this number is **way too small**. 

The number that is shown on the dashboard can be retrieved from the data by adding _all positive values_ in column `cases`. 

In [14]:
print("total cases for Germany:")
# print the sum for all positive values in column "cases" 
print(f' GitHub: {RKI_GITHUB_COVID_INFECTIONS.loc[RKI_GITHUB_COVID_INFECTIONS["cases"] > 0, "cases"].sum():,d}')
# sum of all cases in the data from ARCGIS
print(f' ARCGIS: {RKI_ARCGIS_COVID_BY_DISTRICT["cases"].sum():,d}')

total cases for Germany:
 GitHub: 4,145,852
 ARCGIS: 4,145,852


### Compute Totals

Based on te interpretation of the data described above, the sum of columns `cases`, `deaths` and `recovered` is calculated for each district.

For the sake of convenience, a new dataframe is defined, in which negative values for the number of COVID-19 cases, deaths and recovered patients are set to zero. This is used later for the aggregation of data.

In [15]:
# define dataframe that without the negative values in RKI_GITHUB_COVID_INFECTIONS

RKI_GITHUB_COVID_INFECTIONS_WITHOUT_NEGATIVES = \
    pd.concat([RKI_GITHUB_COVID_INFECTIONS[["district ID", "age group", "sex", "reporting date", "reference date"]], 
              RKI_GITHUB_COVID_INFECTIONS[["cases", "deaths", "recovered"]].apply(lambda a: np.maximum(a,0))], 
              axis="columns")

In [16]:
#  sum up "cases", "deaths", "recovered" for each district
RKI_GITHUB_COVID_BY_DISTRICT_TOTALS = \
    RKI_GITHUB_COVID_INFECTIONS_WITHOUT_NEGATIVES[["district ID", "cases", "deaths", "recovered"]].groupby(by="district ID").sum()

# copy population size for each district from RKI_ARCGIS_COVID_BY_DISTRICT
RKI_GITHUB_COVID_BY_DISTRICT_TOTALS["population"] = RKI_ARCGIS_COVID_BY_DISTRICT["population"]

### Compute Numbers per 100K People 

The sums in columns `cases`, `deaths` and `recovered` are normalized by the population size of the district. 
For the district's population size, the data from the [NPGEO Corona Hub 2020](https://npgeo-corona-npgeo-de.hub.arcgis.com/) is used.

In [17]:
# define a new dataframe by dividing the totals by the population size of each district and multiplying that with 100,000

RKI_GITHUB_COVID_BY_DISTRICT_PER_100K = \
    pd.DataFrame(data=10**5 * RKI_GITHUB_COVID_BY_DISTRICT_TOTALS[["cases", "deaths", "recovered"]].values \
                 / np.array(3 * [RKI_GITHUB_COVID_BY_DISTRICT_TOTALS["population"].values]).T,
                 index=RKI_GITHUB_COVID_BY_DISTRICT_TOTALS.index,
                 columns=["cases per 100k", "deaths per 100k", "recovered per 100k"])

### Compute Totals for the Last Seven Days

The values in columns `cases`, `deaths` and `recovered` of the last seven days are summed up.

In [18]:
# define the date of the earlist data that should be considered
number_of_days = 7 
cut_off_date = np.datetime64(RKI_GITHUB_PUBLICATION_DATE.date() - dt.timedelta(days=number_of_days))

# compute sums for data that has been reported on or after the cut_off_date
RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_TOTALS = \
        RKI_GITHUB_COVID_INFECTIONS_WITHOUT_NEGATIVES.loc[RKI_GITHUB_COVID_INFECTIONS_WITHOUT_NEGATIVES["reporting date"] >= cut_off_date]\
                .groupby(by="district ID").sum()\
                .rename(columns={"cases": "cases last 7 days", "deaths": "deaths last 7 days", "recovered": "recovered last 7 days"})

# ensure that there is data for each district by filling missing data with zeros
# define a dataframe containing the most recent data of each district
df = RKI_GITHUB_COVID_INFECTIONS_WITHOUT_NEGATIVES[["district ID", "reporting date"]].sort_values(by=["district ID", "reporting date"])\
        .groupby(["district ID"]).last()

missing_rows = df.loc[df["reporting date"] < cut_off_date].index.shape[0]
if  missing_rows > 0:
    # there are districts that have not reported data within the last 7 days
    # create dataframe containing the zeroes filling the missing data 
    ddf = pd.DataFrame(data=np.zeros((missing_rows, RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_TOTALS.shape[1]), dtype=np.int64), 
                       index=df.loc[df["reporting date"] < cut_off_date].index,
                       columns=RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_TOTALS.columns)
    # append zeros
    RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_TOTALS = RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_TOTALS.append(ddf)

### Compute Numbers for the Last Seven Days per 100K People 

Normalize the sums for the last seven days by the districts' population size 

In [19]:
# define a new dataframe computing values by dividing the totals for the last seven days by the population size of each district and multiplying it with 100,000
 
RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_PER_100K = \
    pd.DataFrame(data=10**5 * RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_TOTALS.values \
                 / np.array(3 * [RKI_GITHUB_COVID_BY_DISTRICT_TOTALS.loc[RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_TOTALS.index, "population"].values]).T,
                 index=RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_TOTALS.index,
                 columns=["cases last 7 days per 100k", "deaths last 7 days per 100k", "recovered last 7 days per 100k"])

### Create a DataFrame Containing all Values Derived From the COVID-19 Data from GitHub

In [20]:
# combine all of the dataframes with data that has been derived from RKI_GITHUB_COVID_INFECTIONS into one dataframe, 
# only the district names are taken from RKI_ARCGIS_COVID_BY_DISTRICT

RKI_GITHUB_COVID_BY_DISTRICT = \
    pd.concat([RKI_ARCGIS_COVID_BY_DISTRICT["district name"],
               RKI_GITHUB_COVID_BY_DISTRICT_TOTALS, 
               RKI_GITHUB_COVID_BY_DISTRICT_PER_100K, 
               RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_TOTALS,
               RKI_GITHUB_COVID_BY_DISTRICT_LAST_7_DAYS_PER_100K], axis="columns")

## Verify that Derived Data From the [COVID-19 Data from GitHub](https://github.com/robert-koch-institut/SARS-CoV-2_Infektionen_in_Deutschland) and the Data From [NPGEO Corona Hub 2020](https://npgeo-corona-npgeo-de.hub.arcgis.com/) are (More or Less) Identical

In [21]:
EPSILON = 10**-11 # threshold for treating float values as zero
common_columns = list(set(RKI_ARCGIS_COVID_BY_DISTRICT.columns) & set(RKI_GITHUB_COVID_BY_DISTRICT.columns)) 
common_numerical_columns = [c for c in common_columns if RKI_GITHUB_COVID_BY_DISTRICT[c].dtypes != object]

# return True, if all the differences between the absolute values in the common numerical columns is smaller than the threshold

np.max(np.max(np.abs(RKI_ARCGIS_COVID_BY_DISTRICT[common_numerical_columns] - RKI_GITHUB_COVID_BY_DISTRICT[common_numerical_columns]))) < EPSILON

True

## Show Some Data

Based on the result of the verification above, it seems that the interpretation of the data that is provided via GitHub is correct ... or at least identical with what is shown
in the COVID-19 Dashboard. Hence, the following list the most affected districts in Germany can be assumed to be correct.

In [22]:
n = 30
RKI_GITHUB_COVID_BY_DISTRICT.sort_values(by="cases last 7 days per 100k", ascending=False).head(n).style.hide_index().format(FORMAT_MAPPER)

district name,cases,deaths,recovered,population,cases per 100k,deaths per 100k,recovered per 100k,cases last 7 days,deaths last 7 days,recovered last 7 days,cases last 7 days per 100k,deaths last 7 days per 100k,recovered last 7 days per 100k
LK Traunstein,"12,333",218,"11,467","177,485","6,948.8",122.827281,"6,460.8",387,0,2,218.0,0.000,1.1
SK Bremerhaven,"5,619",108,"4,763","113,557","4,948.2",95.106422,"4,194.4",244,1,2,214.9,0.881,1.8
LK Berchtesgadener Land,"7,035",102,"6,594","106,327","6,616.4",95.930479,"6,201.6",211,0,5,198.4,0.000,4.7
SK Neustadt a.d.Weinstraße,"2,116",39,"1,808","53,306","3,969.5",73.162496,"3,391.7",100,0,2,187.6,0.000,3.8
LK Rosenheim,"15,865",468,"14,359","261,721","6,061.8",178.816373,"5,486.4",471,0,3,180.0,0.000,1.1
SK Offenbach,"10,677",185,"9,934","130,892","8,157.1",141.337897,"7,589.5",227,0,0,173.4,0.000,0.0
SK Pforzheim,"8,633",172,"7,896","126,016","6,850.7",136.490604,"6,265.9",205,1,1,162.7,0.794,0.8
SK Rosenheim,"4,613",72,"4,251","63,591","7,254.2",113.223569,"6,684.9",103,1,0,162.0,1.573,0.0
LK Lippe,"19,847",407,"18,724","346,970","5,720.1",117.301208,"5,396.4",554,0,50,159.7,0.000,14.4
SK Salzgitter,"6,313",80,"6,007","103,866","6,078.0",77.022317,"5,783.4",159,0,17,153.1,0.000,16.4


Equally, the following totals for all of Germany appear correct

In [23]:
columns = ["population", "cases", "cases last 7 days", "deaths", "deaths last 7 days", "recovered", "recovered last 7 days"]
RKI_GITHUB_COVID_BY_DISTRICT[columns].sum().to_frame().T.style.hide_index().format(FORMAT_MAPPER)

population,cases,cases last 7 days,deaths,deaths last 7 days,recovered,recovered last 7 days
"83,148,406","4,145,852","59,052","92,971",44,"3,897,819","1,222"


Likewise, the sums for the last 7 days per 100,000 people can be computed

In [24]:
columns = ["cases last 7 days", "deaths last 7 days", "recovered last 7 days"]
(10**5 * RKI_GITHUB_COVID_BY_DISTRICT[columns].sum() / RKI_GITHUB_COVID_BY_DISTRICT["population"].sum())\
    .to_frame().T.rename(columns={c:c+" per 100k" for c in columns}).style.hide_index().format(FORMAT_MAPPER)

cases last 7 days per 100k,deaths last 7 days per 100k,recovered last 7 days per 100k
71.0,0.053,1.5


However, the following numbers, which should reflect the increase of cases, deaths and recovered people during the last day, differ from what the RKI reports. 
This is a mystery ...

In [25]:
columns = ["district ID", "reporting date"]
RKI_GITHUB_COVID_INFECTIONS_WITHOUT_NEGATIVES.sort_values(by=columns).groupby(by=columns).sum()\
    .groupby(by="district ID").last().sum()\
        .to_frame().T.style.hide_index().format(FORMAT_MAPPER)

cases,deaths,recovered
"4,492",1,14


The difference for the last day can also be calculated in a different way: Instead of using the last entry as in the statement above (which may include figures that
more than a day ago, if districts have not reported any data on the previous day), one can also filter by column `reporting date`.

In [26]:
select_timestamp = pd.Timestamp(RKI_GITHUB_PUBLICATION_DATE.date() - dt.timedelta(days=1))
print('the difference between the date of the most recent update and the timestamp for selecting the last day is ' \
        + f'{(RKI_GITHUB_COVID_INFECTIONS_WITHOUT_NEGATIVES["reporting date"].max() - select_timestamp).total_seconds():.0f} seconds')

RKI_GITHUB_COVID_INFECTIONS_WITHOUT_NEGATIVES.loc[RKI_GITHUB_COVID_INFECTIONS_WITHOUT_NEGATIVES["reporting date"] >= select_timestamp, 
                                                  ["cases", "deaths", "recovered"]].sum().to_frame().T.style.hide_index().format(FORMAT_MAPPER)

the difference between the date of the most recent update and the timestamp for selecting the last day is 0 seconds


cases,deaths,recovered
"2,002",1,2


### Compute Totals by Age Group and Sex

In order to close this notebook with something that is not mysterious, the total number of cases, deaths and recovered people is computed by age group and sex. 
This is again in sync with the data of the COVID-19 dashboard.

In [27]:
RKI_GITHUB_COVID_BY_AGE_GROUP_AND_SEX_TOTALS = \
    RKI_GITHUB_COVID_INFECTIONS_WITHOUT_NEGATIVES[["age group", "sex", "cases", "deaths", "recovered"]].groupby(by=["age group", "sex"]).sum()
RKI_GITHUB_COVID_BY_AGE_GROUP_AND_SEX_TOTALS.style.format(FORMAT_MAPPER)